In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

import requests
import csv
import os

import numpy as np
import pandas as pd
from numpy import random
from time import sleep

from BaTparser import Parser

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome()
driver.get('https://bringatrailer.com/toyota/land-cruiser-fj60-fj62/')

In [ ]:
# extend page
while True:
    try:
        button = driver.find_element(By.XPATH , "//div[@class='auctions-footer auctions-footer-previous']/*")
        button.click()
    except NoSuchElementException:
        break
    sleeptime = random.uniform(3, 5)
    sleep(sleeptime)

In [ ]:
html = driver.page_source
html_soup = BeautifulSoup(html, 'lxml')

listings = html_soup.find('div', {'class': 'blocks'}).find_all('div', {'class': 'block'})
links = []
for listing in listings:
    link = listing.a.get('href')
    links.append(link)
# remove below link because it was REMOVED from auction, but page still exists (i.e., auction did not finish)
links.remove('https://bringatrailer.com/listing/1984-toyota-land-cruiser-fj60-7/')

In [ ]:
parser = Parser()
with open ('landcruisers.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    # header
    csv_writer.writerow(['title','price','closing_date', 'result', 'no_views','no_watchers','no_comments', 'no_bids', 'seller_state', 
                         'year', 'series', 'miles','engine','trans','exterior','interior', 'misc', 'url'])
    for url in links[:382]:
        # Retrieve source page
        source = requests.get(url).text
        # Scrape page
        scraped_page = parser.parse_page(source)
        scraped_page.append(url)
        csv_writer.writerow(scraped_page)
        # Sleepy time
        sleeptime = random.uniform(10, 90)
        sleep(sleeptime)